In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/high/'

In [2]:
file_name = 'normalized_to_rating_filter_track_20_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_20_user_100.csv'
pop_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))
pop_rating = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_rating_top_N_' + file_name[:-3] + 'pkl'))

num_user = len(svd['uid'].unique())
num_user

945

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [5]:
svd['count'] = 1
svd

,uid,tid,rating,count
0,0,3,2.609116,1
1,0,5,2.148478,1
2,0,8,2.397060,1
3,0,11,1.915828,1
4,0,12,2.141889,1
...,...,...,...,...
35922656,944,39083,2.324390,1
35922657,944,39084,1.955682,1
35922658,944,39085,1.734721,1
35922659,944,39086,2.189229,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count

,tid,count,rating
46,46,8598,1469.00
647,647,3861,734.25
2690,2690,3451,534.50
1582,1582,3392,958.50
2611,2611,3209,740.75
...,...,...,...
16900,16900,6,5.25
15438,15438,5,5.00
31362,31362,5,1.25
39024,39024,5,2.50


In [7]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized

,tid,count,rating
46,46,8598,5.000000
647,647,3861,2.245290
2690,2690,3451,2.006862
1582,1582,3392,1.972552
2611,2611,3209,1.866132
...,...,...,...
16900,16900,6,0.003489
15438,15438,5,0.002908
31362,31362,5,0.002908
39024,39024,5,0.002908


In [73]:
alpha = 0.8
top_n_size = 4
top_n_lists_g = []
for groups in tqdm(groups_n):
    
    group_size = len(groups[0])
    top_n_lists = []
    
    for group in groups:
        
        predictions = []
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = svd[svd['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            predictions.append(prediction)
        rating_table['rating'] /= group_size 
        rating_table['rating'] *= alpha
        pop = pop_normalized.copy()
        pop['rating'] *= (1 - alpha)
        rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
#         rating_table = rating_table[rating_table['count'] == group_size]
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)                                   

In [74]:
threshold = 2.25
for i in tqdm(range(len(groups_n))):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    precisions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_rating = 0
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_rating += 1
                    break
        
        precision = high_rating / top_n_size
        precisions.append(precision)
        
    avg_precision = 0
    for precision in precisions:
        avg_precision += precision
    avg_precision /= len(precisions)
    print(avg_precision)

0.4867584745762712
0.610128617363344
0.6590909090909091
0.7123893805309734



In [195]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        

0.4384689835799772
0.34093950544457624
0.3358646172870294
0.33769451038646087

